# Systemy uczące się - Zad. dom. 1: Minimalizacja ryzyka empirycznego
Celem zadania jest zaimplementowanie własnego drzewa decyzyjnego wykorzystującego idee minimalizacji ryzyka empirycznego. 

### Autor rozwiązania
Uzupełnij poniższe informacje umieszczając swoje imię i nazwisko oraz numer indeksu:

In [56]:
NAME = "Bartłomiej Andree"
ID = "162961"

## Twoja implementacja

Twoim celem jest uzupełnić poniższą klasę `TreeNode` tak by po wywołaniu `TreeNode.fit` tworzone było drzewo decyzyjne minimalizujące ryzyko empiryczne. Drzewo powinno wspierać problem klasyfikacji wieloklasowej (jak w przykładzie poniżej). Zaimplementowany algorytm nie musi (ale może) być analogiczny do zaprezentowanego na zajęciach algorytmu dla klasyfikacji. Wszelkie przejawy inwencji twórczej wskazane. **Pozostaw komenatrze w kodzie, które wyjaśniają Twoje rozwiązanie.**

Schemat oceniania:
- wynik na zbiorze Iris (automatyczna ewaluacja) celność klasyfikacji >= prostego baseline'u + 10%: +40%,
- wynik na ukrytym zbiorze testowym 1 (automatyczna ewaluacja) celność klasyfikacji >= prostego baseline'u + 15%: +30%,
- wynik na ukrytym zbiorze testowym 2 (automatyczna ewaluacja) celność klasyfikacji >= prostego baseline'u + 5%: +30%.

Niedozwolone jest korzystanie z zewnętrznych bibliotek do tworzenia drzewa decyzyjnego (np. scikit-learn). 
Możesz jedynie korzystać z biblioteki numpy.

#### Uwaga: Możesz dowolnie modyfikować elementy tego notebooka (wstawiać komórki i zmieniać kod), o ile będzie się w nim na koniec znajdowała kompletna implementacja klasy `TreeNode` w jednej komórce.

In [57]:
import numpy as np

class TreeNode:
    def __init__(self):
        self.left: TreeNode | None = None   # wierzchołek znajdujący się po lewej stronie
        self.right: TreeNode | None = None  # wierzchołek znajdujący się po prawej stronie
        self.value = None                   # wartość liścia (przypisana odpowiedź)
        self.split_feature = None           # indeks cechy, po której dzielimy
        self.split_threshold = None         # próg podziału

    def _gini(self, y):
        """
        Funkcja obliczająca współczynnik Gini dla zbioru etykiet y.
        Współczynnik Gini mierzy stopień zanieczyszczenia zbioru - im mniejsza wartość,
        tym czystszy zbiór (więcej przykładów z tej samej klasy).
        """
        # Dla pustego zbioru zwracamy 0, bo nie ma w nim zanieczyszczenia
        if len(y) == 0:
            return 0

        # Zliczamy ile mamy przykładów z każdej klasy
        classes, counts = np.unique(y, return_counts=True)

        # Obliczamy prawdopodobieństwo wystąpienia każdej z klas
        # To po prostu liczba przykładów danej klasy podzielona przez liczbę wszystkich przykładów
        probabilities = counts / len(y)

        # Wzór na współczynnik Gini: 1 - suma(p_i^2)
        # 1 minus suma kwadratów prawdopodobieństw dla każdej klasy
        # Im mniejszy jest współczynnik Gini, tym lepiej - bardziej jednorodny zbiór
        gini = 1 - np.sum(probabilities ** 2)

        return gini

    def _find_split(self, data, target):
        """
        Szuka najlepszego podziału danych według kryterium Giniego.
        Przeszukuje wszystkie cechy i wszystkie możliwe wartości progów podziału,
        wybierając ten podział, który daje najmniejszy ważony współczynnik Gini.
        To oznacza, że dzieli dane na najbardziej homogeniczne podzbiory.
        """
        # Zaczynamy od najgorszego możliwego podziału (nieskończony Gini)
        best_gini = float('inf')
        best_feature = None
        best_threshold = None

        n_samples, n_features = data.shape

        # Sprawdzamy każdą cechę jako potencjalny kandydat do podziału
        for feature in range(n_features):
            # Bierzemy wszystkie unikalne wartości tej cechy jako potencjalne progi podziału
            # Nie ma sensu sprawdzać wartości, które się powtarzają, bo dadzą ten sam podział
            unique_values = np.unique(data[:, feature])

            # Testujemy każdą unikalną wartość jako próg
            for threshold in unique_values:
                # Tworzymy maski boolowskie dla lewej i prawej strony podziału
                # Lewa strona to wszystkie przykłady, gdzie wartość cechy ≤ progu
                left_mask = data[:, feature] <= threshold
                # Prawa strona to wszystkie pozostałe przykłady
                right_mask = ~left_mask

                # Jeśli któraś strona jest pusta, to podział jest bezużyteczny
                # (nie dzieli zbioru na dwie części). Pomijamy taki podział.
                if np.sum(left_mask) == 0 or np.sum(right_mask) == 0:
                    continue

                # Obliczamy współczynnik Gini dla obu stron podziału
                gini_left = self._gini(target[left_mask])
                gini_right = self._gini(target[right_mask])

                # Obliczamy średni ważony współczynnik Gini dla całego podziału
                # Waga każdej strony zależy od liczby przykładów po tej stronie
                n_left = np.sum(left_mask)
                n_right = np.sum(right_mask)
                gini_split = (n_left/n_samples) * gini_left + (n_right/n_samples) * gini_right

                # Jeśli ten podział daje lepszy (mniejszy) współczynnik Gini niż dotychczasowy najlepszy,
                # to zapamiętujemy go jako nowy najlepszy podział
                # wynika to z faktu, że minimalizujemy ryzyko empiryczne
                if gini_split < best_gini:
                    best_gini = gini_split
                    best_feature = feature
                    best_threshold = threshold

        # Zwracamy indeks najlepszej cechy, najlepszy próg i wartość Gini dla tego podziału
        return best_feature, best_threshold, best_gini

    def fit(self, data: np.ndarray, target: np.ndarray) -> None:
        """
        Trenuje drzewo decyzyjne używając algorytmu minimalizacji ryzyka empirycznego.
        Rekurencyjnie dzieli dane na podstawie kryterium Giniego, aby stworzyć drzewo
        klasyfikujące przykłady do odpowiednich klas.

        Args:
            data (np.ndarray): macierz cech o wymiarach (n, m), gdzie n to liczba przykładów, a m to liczba cech
            target (np.ndarray): wektor klas o długości n, gdzie n to liczba przykładów
        """


        # Poniej znajdziesz przykładowy "pseudo-kod" rozwiązania, nie musisz się go trzymać
		# (możesz zaimplementować to w inny sposób, jeżeli wolisz)
		#
		# Znajdź najlepszy podział x, y
		# if uzyskano poprawę funkcji celu (bądź inny, zaproponowany przez Ciebie warunek):
		# 	podziel dane na dwie części data_left i data_right, zgodnie z warunkiem
		# 	self.left = Node()
		# 	self.right = Node()
		# 	self.left.fit(data_left)
		# 	self.right.fit(data_right)
		# else:
		# 	obecny Node jest liściem, zapisz jego odpowiedź


        # Jeśli wszystkie przykłady należą do tej samej klasy, tworzymy liść z tą klasą jako wartością
        if len(np.unique(target)) == 1:
            # Wartość liścia to klasa wszystkich przykładów w tym węźle
            self.value = target[0]
            return

        # Szukamy najlepszego podziału minimalizującego współczynnik Gini - czyli minimalizującego ryzyko empiryczne
        best_feature, best_threshold, _ = self._find_split(data, target)

        # Jeśli nie udało się znaleźć sensownego podziału, tworzymy liść z najczęstszą klasą jako wartością
        if best_feature is None:
            # Używamy np.bincount do znalezienia najczęstszej klasy
            # argmax zwraca indeks największej wartości w tablicy
            self.value = np.bincount(target).argmax()
            return

        # Jeśli znaleźliśmy dobry podział, zapisujemy jego parametry
        self.split_feature = best_feature
        self.split_threshold = best_threshold

        # Dzielimy dane na dwa podzbiory według znalezionego podziału
        left_mask = data[:, best_feature] <= best_threshold
        right_mask = ~left_mask

        # Dodatkowe sprawdzenie, czy podział faktycznie dzieli dane
        # (raczej niepotrzebne, bo sprawdzamy to wcześniej, ale dla pewności)
        if np.sum(left_mask) == 0 or np.sum(right_mask) == 0:
            # Jeśli nie dzieli, tworzymy liść zamiast podwęzłów
            self.value = np.bincount(target).argmax()
            return

        # Tworzymy lewy i prawy węzeł dziecka
        self.left = TreeNode()
        self.right = TreeNode()

        # Rekurencyjnie trenujemy lewe i prawe poddrzewo na odpowiednich podzbiorach danych
        # Tu dzieje się główna idea budowy drzewa - dziel i zwyciężaj
        self.left.fit(data[left_mask], target[left_mask])
        self.right.fit(data[right_mask], target[right_mask])

    def predict(self, data: np.ndarray) -> np.ndarray:
        """
        Przewiduje klasy dla nowych danych na podstawie wytrenowanego drzewa.
        Przechodzi drzewo dla każdego przykładu i zwraca klasę z odpowiedniego liścia.

        Args:
            data (np.ndarray): macierz cech o wymiarach (n, m), gdzie n to liczba przykładów, a m to liczba cech

        Returns:
            np.ndarray: wektor przewidzianych klas o długości n, gdzie n to liczba przykładów
        """


        # Poniżej znajdziesz przykładowy "pseudo-kod" rozwiązania, nie musisz się go trzymać
		# (możesz zaimplementować to w inny sposób, jeżeli wolisz),
		# ważne by metoda TreeNode.predict zwracała wektor przewidzianych klas
		#
        # Dla każdego przykładu w data:
		#   node = self
        #   if node nie jest liściem:
        #       if warunek podziału node jest spełniony:
        #           node = node.right
        #       else:
        #           node = node.left
        #   y_pred[i] = zwróć wartość node (liść)


        # Inicjujemy wektor predykcji zerami
        y_pred = np.zeros(data.shape[0], dtype=int)

        # Dla każdego przykładu z danych wejściowych...
        for i, sample in enumerate(data):
            # Zaczynamy od korzenia drzewa
            node = self

            # Przechodzimy w dół drzewa, dopóki nie dotrzemy do liścia
            # Liść to węzeł, który ma przypisaną konkretną wartość klasy
            while node.value is None:
                # Sprawdzamy warunek podziału dla bieżącego węzła
                if sample[node.split_feature] <= node.split_threshold:
                    # Jeśli wartość cechy jest mniejsza lub równa progowi, idziemy w lewo
                    node = node.left
                else:
                    # W przeciwnym razie idziemy w prawo
                    node = node.right

            # Gdy dotrzemy do liścia, przypisujemy jego wartość jako predykcję dla tego przykładu
            y_pred[i] = node.value

        return y_pred

## Przykład trenowanie i testowania drzewa
 
Później znajduje się przykład trenowania i testowania drzewa na zbiorze danych `iris`, który zawierający 150 próbek irysów, z czego każda próbka zawiera 4 atrybuty: długość i szerokość płatków oraz długość i szerokość działki kielicha. Każda próbka należy do jednej z trzech klas: `setosa`, `versicolor` lub `virginica`, które są zakodowane jak int.

Możesz go wykorzystać do testowania swojej implementacji. Możesz też zaimplementować własne testy lub użyć innych zbiorów danych, np. innych [zbiorów danych z scikit-learn](https://scikit-learn.org/stable/datasets/toy_dataset.html#toy-datasets).

In [58]:
#!pip install scikit-learn

In [59]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.33, random_state=2024)

tree_model = TreeNode()
tree_model.fit(X_train, y_train)
y_pred = tree_model.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.84
